# Convolution Neural Network (1)
* convolution
* filter, pooling

* [강의출처](https://www.youtube.com/watch?v=rySyghVxo6U&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=19)
* [강의출처](https://www.youtube.com/watch?v=wQtkdq3tmJ8&list=PLQ28Nx3M4JrhkqBVIXg-i5_CVVoS1UzAv&index=20)
* [코드출처](https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_1_mnist_cnn.ipynb)

## Library Load

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# mnist dataset
import torchvision.datasets as dsets
import torchvision.transforms as transforms

## Environment Setting

In [4]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# set seed
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)

## Parameter Setting

In [34]:
learning_rate = 0.001
drop_rate = 0
nb_epochs = 15
batch_size = 100

## Dataset loading

In [6]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=False)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

In [7]:
# dataset loader
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

## Model

In [76]:
class MnistCnnClassifier(nn.Module):
    def __init__(self):
        super(MnistCnnClassifier, self).__init__()
        
        # layer1
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        
        # layer 2
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        
        # fc 
        self.fc = nn.Linear(64*7*7, 10, bias=True)
        nn.init.xavier_uniform_(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        
        out = self.fc(out)
        return out

In [77]:
model = MnistCnnClassifier().to(device)

In [78]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [79]:
train_batch = len(train_loader)

for epoch in range(nb_epochs):
    
    avg_cost = 0
    
    for X, Y, in train_loader:
        X = X.to(device)
        Y = Y.to(device)
                
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / train_batch
            
    print("[Epoch:{}] Cost: {}".format(epoch+1, avg_cost))

[Epoch:1] Cost: 0.15279115736484528
[Epoch:2] Cost: 0.049703218042850494
[Epoch:3] Cost: 0.034438032656908035
[Epoch:4] Cost: 0.027119629085063934
[Epoch:5] Cost: 0.02052033320069313
[Epoch:6] Cost: 0.016875218600034714
[Epoch:7] Cost: 0.013076558709144592
[Epoch:8] Cost: 0.011476566083729267
[Epoch:9] Cost: 0.00845374632626772
[Epoch:10] Cost: 0.006385910324752331
[Epoch:11] Cost: 0.0059549943543970585
[Epoch:12] Cost: 0.005469640251249075
[Epoch:13] Cost: 0.004523065406829119
[Epoch:14] Cost: 0.005021876655519009
[Epoch:15] Cost: 0.004349771421402693


In [83]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9800000190734863
